In [7]:
import string
from azlyrics_scrapping import create_or_open_index
from sklearn.metrics.pairwaise import cosine_similarity

ModuleNotFoundError: No module named 'sklearn'

In [4]:
index = create_or_open_index('./index')
all_docs = index.searcher().documents() 
all_lyrics = list()
for doc in all_docs:
    all_lyrics.append(doc['full_lyrics'])

In [6]:
len(all_lyrics)

123